In [1]:
import json
import math
import pandas as pd

# Load the JSON data

json_file_path = r'C:\Users\mueller15\OneDrive - Avaya\AXP Analytics\DialogFlow\bquxjob_1badbb0_187b38bd236.json'

with open(json_file_path, 'r') as j:
    data = json.loads(j.read())


# Create an empty list to hold the parsed data
parsed_data = []

# Loop through each conversation turn and extract the relevant information
for turn in data:
    conversation_name = turn['conversation_name'].split("/").pop()
    turn_position = turn['turn_position']
    request_time = turn['request_time'].split(" UTC")[0]
    language_code = turn['language_code']
    request = json.loads(turn['request'])
    response = json.loads(turn['response'])
    derived_data = json.loads(turn['derived_data'])
    conversation_signals = json.loads(turn['conversation_signals'])

    # Create a dictionary with the parsed data for this conversation turn
    parsed_turn = {
        'conversation_name': conversation_name,
        'turn_position': turn_position,
        'request_time': request_time,
        'language_code': language_code,
        'request': request,
        'response': response,
        'derived_data': derived_data,
        'conversation_signals': conversation_signals
    }

    # Append the parsed data for this conversation turn to the list of parsed data
    parsed_data.append(parsed_turn)

# Create a pandas DataFrame from the parsed data, with a unique identifier column
df = pd.DataFrame(parsed_data)
df['unique_identifier'] = df['conversation_name'] + '_' + df['turn_position']

def flatten_json_columns(df):
    """
    Flatten all columns that contain JSON data into new columns.
    """
    json_columns = []
    for column in df.columns:
        if df[column].apply(lambda x: isinstance(x, (dict, list))).all():
            json_columns.append(column)

    for column in json_columns:
        column_data = df[column].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
        flattened_data = pd.json_normalize(column_data)
        flattened_data.columns = [f"{column}_{col}" for col in flattened_data.columns]
        df = pd.concat([df, flattened_data], axis=1)
        df = df.drop(column, axis=1)

    return df

def compute_latency(latency_list):
    latency = latency_list[0].get('nanos')/1000000000 + float(latency_list[0].get('seconds') or 0)
    return latency
        
    
df = flatten_json_columns(df)
df['response_queryResult.webhookLatencies'] = df['response_queryResult.webhookLatencies'].map(lambda x: compute_latency(x) if type(x) == list else x)


In [ ]:
df.to_csv(r'C:\Users\mueller15\OneDrive - Avaya\AXP Analytics\DialogFlow\bquxjob_1badbb0_187b38bd236.csv', index=False)